# Project 2: **Creating the Corpus, Tf-idf, Linear Model** using `BucketIterator`

In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize

In [2]:
df = pd.read_csv('DataFrames/final_pfam_dataframe_no_ID.csv')
df.head()

,Response,BGC,PF00218,PF00291,PF00290,PF02146,PF01649,PF02518,PF00512,PF00486,...,PF15902,PF09822,PF03739,PF13690,PF04509,PF10639,PF04439,PF12412,PF16347,PF02995
0,NonResponder,ERS2034069_k141_12424,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NonResponder,ERS2034069_k141_46673,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NonResponder,ERS2034069_k141_61665,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NonResponder,ERS2034070_k141_24121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NonResponder,ERS2034071_k141_39888,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
len(df.columns)

1458

In [4]:
l = df[df.columns[2:]].sum()

len(l[l>0])

1456

In [5]:
len(df[df.Response == 'Responder'])

106

In [6]:
len(df[df.Response == 'NonResponder'])

338

### Corpus is a large Strings for all rows 
> One for Responder

> Another one for NonResponder

In [7]:
def corpus_maker(df, response):
    corpus = ""
    cols = df.columns[2:]
    for index in range(len(df)):
        col_names = ' '
        if df.loc[index, 'Response'] == response:
            for col in cols:
                if df.loc[index, col] == 1:
                    col_names += " " + col
            corpus += col_names + "."
    return corpus

In [15]:
responder = corpus_maker(df, 'Responder')


In [16]:
responder[:1000]

'  PF00005 PF08659 PF00501 PF00109 PF02801 PF00550 PF12833 PF00165 PF07690 PF00668 PF00664 PF13649 PF08241 PF00589 PF16197 PF00698 PF08242 PF00425 PF00108 PF06167 PF01135.  PF03176 PF02272 PF13561 PF00106 PF08659 PF00501 PF00109 PF02801 PF00550 PF13279 PF03061 PF03548 PF13723 PF13489 PF00891 PF03279 PF00221 PF00326 PF00535 PF01553 PF01738 PF07859 PF12837 PF00037 PF12838 PF13187 PF13237 PF12797 PF12800 PF08242 PF13481 PF01551 PF04060 PF02508 PF04205 PF03116 PF01411 PF07973 PF13375 PF01512 PF10531 PF03796 PF00892 PF00772 PF01784 PF02591 PF11306 PF04246.  PF13561 PF00106 PF08659 PF00501 PF00109 PF02801 PF00550 PF00975 PF13193 PF00668 PF00126 PF03466 PF01554 PF03721 PF00589 PF03446 PF00881 PF02771 PF00441 PF08028 PF16197 PF02737 PF00725 PF00698 PF08240 PF00107 PF01425 PF00144 PF08020 PF01709 PF13602 PF14765.  PF00512 PF00486 PF00072 PF03176 PF00501 PF00593 PF07715 PF05977 PF07690 PF01032 PF00975 PF00668 PF03621 PF00756 PF11806 PF14905 PF00924 PF00672 PF02321 PF11604 PF00529 PF16576 PF16572

In [17]:
non_responder = corpus_maker(df, 'NonResponder')

In [18]:
non_responder[:1000]

'  PF00005 PF00501 PF00550 PF01522 PF00975 PF13193 PF00668 PF01408 PF03773 PF03435 PF00145 PF02492 PF01475 PF01297 PF00950 PF07683.  PF03176 PF13561 PF00106 PF08659 PF00501 PF00109 PF02801 PF00550 PF00890 PF13450 PF01593 PF13279 PF03061 PF07977 PF03548 PF01522 PF13489 PF00891 PF00221 PF00535 PF03417 PF01553 PF13411 PF00376 PF09835 PF05175 PF13847 PF13649 PF08241 PF08242 PF01551 PF01177.  PF00501 PF00550 PF00668 PF03023 PF01554 PF14667 PF00702 PF01943 PF13412 PF12802 PF01325 PF03591 PF05437 PF08282 PF13581 PF01740 PF13466 PF12320 PF13558 PF00122 PF13630 PF07853 PF12840 PF01022 PF04464.  PF00501 PF00109 PF02801 PF00550 PF12833 PF00165 PF00975 PF13193 PF00668 PF01648 PF16197 PF12697.  PF03176 PF00654 PF13561 PF00106 PF08659 PF00109 PF02801 PF00550 PF00890 PF13450 PF01593 PF13279 PF03061 PF03548 PF01522 PF13723 PF08545 PF13489 PF00891 PF03279 PF00221 PF00535 PF01553 PF09835 PF05175 PF13847 PF13649 PF08241 PF00571 PF08242 PF03772 PF13567 PF00834 PF02911 PF00551 PF01300.  PF13561 PF00106 PF0

In [19]:
len(responder), len(non_responder)

(21948, 78804)

### `responder` and `non_responder` are 2 long strings

### Create articles
- Each article is a string consisits of all column names which have value = 1
- Each row represent 1 article 

In [20]:
# Create NonResponder articels

from nltk.tokenize import word_tokenize, sent_tokenize

sents=sent_tokenize(non_responder)
for index, sent in enumerate(sents):
    sent = sent.replace('.', "")
    text_file = open(r"corpus_new\NonResponder\%d.txt" % index, "w")
    text_file.write(sent)
    text_file.close()

In [21]:
# Create Responder articels

from nltk.tokenize import word_tokenize, sent_tokenize

sents=sent_tokenize(responder)
for index, sent in enumerate(sents):
    sent = sent.replace('.', "")
    text_file = open(r"corpus_new\Responder\%d.txt" % index, "w")
    text_file.write(sent)
    text_file.close()

### Let's check if there's any duplicate 
- There shouldn't be any

In [22]:
sents=sent_tokenize(responder)
ll= []
for index, sent in enumerate(sents):
    l = []
    sent = sent.replace('.', "")
    l.append(sent.split(" "))
    ll.extend(l)
    

In [23]:
from collections import Counter
Counter(ll[40])

Counter({'PF13561': 1,
         'PF00106': 1,
         'PF08659': 1,
         'PF00109': 1,
         'PF02801': 1,
         'PF00890': 1,
         'PF13450': 1,
         'PF01593': 1,
         'PF13279': 1,
         'PF03061': 1,
         'PF08281': 1,
         'PF00070': 1,
         'PF03807': 1,
         'PF04545': 1,
         'PF00588': 1,
         'PF01946': 1,
         'PF13173': 1,
         'PF00436': 1,
         'PF01488': 1,
         'PF13635': 1,
         'PF11026': 1,
         'PF08501': 1,
         'PF00709': 1})

### Let's load the data

In [8]:
from sklearn.datasets import load_files

# for reproducibility
random_state = 42

DATA_DIR = "./corpus_new/"
data = load_files(DATA_DIR, encoding="utf-8", decode_error="replace", random_state=random_state)

### `data` is a bunch object consists of `'data'` and `'target'`

In [9]:
import pandas as pd

df_corpus = pd.DataFrame(list(zip(data['data'], data['target'])), columns=['text', 'label'])
print(f"we have {len(df_corpus)} articles in our dataset")
df_corpus.head(10)

we have 444 articles in our dataset


,text,label
0,PF02518 PF00512 PF13561 PF00106 PF08659 PF0050...,0
1,PF07728 PF00004 PF13165 PF13353 PF04055 PF1339...,0
2,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,0
3,PF00005 PF13561 PF00106 PF08659 PF00501 PF0010...,1
4,PF00440 PF08541 PF01551 PF01613 PF03050 PF0162...,0
5,PF09950 PF11681 PF11863 PF13262 PF04233 PF1420...,0
6,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,1
7,PF13165 PF13353 PF04055 PF13450 PF00152 PF0133...,0
8,PF03176 PF13561 PF00106 PF08659 PF00501 PF0010...,0
9,PF13165 PF13353 PF04055 PF00890 PF13450 PF0029...,1


In [10]:
from collections import Counter

# Let's look at the 1st row of data
Counter(df_corpus.iloc[0,0].replace(".", " ").split(" "))

Counter({'PF02518': 1,
         'PF00512': 1,
         'PF13561': 1,
         'PF00106': 1,
         'PF08659': 1,
         'PF00501': 1,
         'PF00109': 1,
         'PF02801': 1,
         'PF00550': 1,
         'PF00890': 1,
         'PF13450': 1,
         'PF01593': 1,
         'PF13279': 1,
         'PF03061': 1,
         'PF13970': 1,
         'PF07977': 1,
         'PF03548': 1,
         'PF01522': 1,
         'PF04383': 1,
         'PF13723': 1,
         'PF08545': 1,
         'PF13489': 1,
         'PF00891': 1,
         'PF03279': 1,
         'PF01370': 1,
         'PF00221': 1,
         'PF09190': 1,
         'PF01406': 1,
         'PF09334': 1,
         'PF01553': 1,
         'PF09835': 1,
         'PF05175': 1,
         'PF13847': 1,
         'PF13649': 1,
         'PF08241': 1,
         'PF08447': 1,
         'PF01590': 1,
         'PF08242': 1})

In [11]:
len(list(set(df_corpus.iloc[0,0].split(" "))))

38

In [20]:
pfams  = []
for txt in df_corpus["text"]:
    pfams.extend(txt.split(" "))
    
# Total number of pfams in our dataset
print(f"Total number of pfams in our dataset {len(pfams)}")

Total number of pfams in our dataset 12487


In [21]:
unique_pfams = list(set(pfams))

print(f"Total number of unique pfams in our dataset {len(unique_pfams)}")

Total number of unique pfams in our dataset 1457


### Save and load the corpus dataframe

In [ ]:
df_corpus.to_csv("DataFrames/df_corpus_pfams.csv", index=False)

In [420]:
df_corpus = pd.read_csv("DataFrames/df_corpus_pfams.csv")
df_corpus.head()                      

,text,label
0,PF02518 PF00512 PF13561 PF00106 PF08659 PF0050...,0
1,PF07728 PF00004 PF13165 PF13353 PF04055 PF1339...,0
2,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,0
3,PF00005 PF13561 PF00106 PF08659 PF00501 PF0010...,1
4,PF00440 PF08541 PF01551 PF01613 PF03050 PF0162...,0


In [421]:
len(df_corpus[ df_corpus.label==1])

106

In [422]:
len(df_corpus[df_corpus.label==0])

338

### Different rows has different length for `text` column

In [423]:
len(df_corpus.iloc[1, :][0]), len(df_corpus.iloc[0, :][0])

(183, 303)

### The number of Responders and NonResponders matches in `df_corpus` and the original `df`

> label 0 &#8594; NonResponder

> Labe 1 &#8594; Responder

### Let's create 2 seperate dataframes for test and train

In [424]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_corpus, test_size = 0.3, random_state = 42, stratify = df_corpus.label)

In [425]:
train.head()

,text,label
36,PF02518 PF00512 PF00072 PF03176 PF04616 PF1356...,0
411,PF09950 PF11681 PF11863 PF13262 PF09979 PF0423...,1
102,PF02518 PF00512 PF00072 PF04616 PF13561 PF0010...,1
135,PF00072 PF00501 PF13450 PF12833 PF00165 PF1319...,1
199,PF07690 PF13302 PF00926 PF07992 PF02535 PF0008...,0


In [426]:
round(len(train[train.label == 1])/ len(train),2) ,round(len(test[test.label == 1])/len(test), 2)

(0.24, 0.24)

In [427]:
train.reset_index(drop = True), test.reset_index(drop=True)

(                                                  text  label
 0    PF02518 PF00512 PF00072 PF03176 PF04616 PF1356...      0
 1    PF09950 PF11681 PF11863 PF13262 PF09979 PF0423...      1
 2    PF02518 PF00512 PF00072 PF04616 PF13561 PF0010...      1
 3    PF00072 PF00501 PF13450 PF12833 PF00165 PF1319...      1
 4    PF07690 PF13302 PF00926 PF07992 PF02535 PF0008...      0
 ..                                                 ...    ...
 305  PF00072 PF00501 PF13450 PF12833 PF00165 PF1319...      0
 306  PF13165 PF13353 PF04055 PF01276 PF00860 PF1280...      0
 307  PF00072 PF00501 PF00109 PF02801 PF00550 PF1450...      1
 308  PF07728 PF00004 PF00654 PF13561 PF00106 PF0865...      0
 309  PF13165 PF13353 PF04055 PF13394 PF00528 PF0490...      1
 
 [310 rows x 2 columns],
                                                   text  label
 0    PF00072 PF00501 PF00550 PF00890 PF01370 PF0066...      0
 1    PF00583 PF00717 PF13508 PF13673 PF11799 PF0081...      0
 2    PF07728 PF00004 PF0065

In [ ]:
train.to_csv('DataFrames/train_pfam_corpus_p4.csv', index = False)
test.to_csv('DataFrames/test_pfam_corpus_p4.csv', index = False)

In [429]:
df_train = pd.read_csv('DataFrames/train_pfam_corpus_p4.csv')
df_test = pd.read_csv('DataFrames/test_pfam_corpus_p4.csv')

In [430]:
df_train.head()

,text,label
0,PF02518 PF00512 PF00072 PF03176 PF04616 PF1356...,0
1,PF09950 PF11681 PF11863 PF13262 PF09979 PF0423...,1
2,PF02518 PF00512 PF00072 PF04616 PF13561 PF0010...,1
3,PF00072 PF00501 PF13450 PF12833 PF00165 PF1319...,1
4,PF07690 PF13302 PF00926 PF07992 PF02535 PF0008...,0


In [431]:
df_test.head()

,text,label
0,PF00072 PF00501 PF00550 PF00890 PF01370 PF0066...,0
1,PF00583 PF00717 PF13508 PF13673 PF11799 PF0081...,0
2,PF07728 PF00004 PF00654 PF00005 PF13561 PF0010...,1
3,PF00072 PF00005 PF00501 PF13450 PF13193 PF0277...,1
4,PF00218 PF00291 PF00290 PF02146 PF01649 PF0251...,1


In [432]:
type(test.iloc[0].text), type(test.iloc[0].label)

(str, numpy.int64)

---

## Let's explore `tf-idf`
- Each word in the corpus gets a value based on it's accurance and frequency
- Each article (row) is an array of shape `(1, vocab_size)`

In [433]:
# articles_train is a 1D numpy  array of all train articels 

articles_train = df_train.text.values
print( type(articles_train), len(articles_train) )

<class 'numpy.ndarray'> 310


In [434]:
# 1000 characters of the 1st article in the train dataframe
articles_train[0][:1000]

'PF02518 PF00512 PF00072 PF03176 PF04616 PF13561 PF00106 PF08659 PF00109 PF02801 PF00550 PF00890 PF13450 PF01593 PF13279 PF03061 PF13620 PF13715 PF13970 PF07977 PF03548 PF01522 PF04383 PF13723 PF08545 PF13489 PF00891 PF03279 PF01370 PF00221 PF09190 PF01406 PF09334 PF07523 PF07980 PF14322 PF00593 PF07715 PF10566 PF14508 PF12833 PF00165 PF07495 PF07494 PF00535 PF13641 PF01553 PF13704 PF09835 PF05175 PF13847 PF13649 PF08241 PF08242 PF05401 PF13589'

In [435]:
articles_test = df_test.text.values
len(articles_test)

134

In [436]:
labels_train = df_train.label.values
print( type(labels_train), len(labels_train) )

<class 'numpy.ndarray'> 310


In [437]:
# Differenet rows has different length
len(articles_train[0]), len(articles_train[1])

(447, 79)

In [438]:
labels_train[0]

0

In [439]:
labels_test = df_test.label.values
print( type(labels_test), len(labels_test) )

<class 'numpy.ndarray'> 134


In [440]:
df_corpus.head()

,text,label
0,PF02518 PF00512 PF13561 PF00106 PF08659 PF0050...,0
1,PF07728 PF00004 PF13165 PF13353 PF04055 PF1339...,0
2,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,0
3,PF00005 PF13561 PF00106 PF08659 PF00501 PF0010...,1
4,PF00440 PF08541 PF01551 PF01613 PF03050 PF0162...,0


In [441]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [442]:
articles = df_corpus.text.values
X = tfidf_vectorizer.fit_transform(articles)
X

<444x1456 sparse matrix of type '<class 'numpy.float64'>'
	with 12483 stored elements in Compressed Sparse Row format>

In [443]:
type(X)

scipy.sparse.csr.csr_matrix

In [444]:
# the 1st article in dataset
X[0].shape

(1, 1456)

In [445]:
print(X[0])

  (0, 986)	0.16538262692914568
  (0, 383)	0.20878944068577385
  (0, 1003)	0.2439856140402456
  (0, 985)	0.15414685198581074
  (0, 1324)	0.15414685198581074
  (0, 1351)	0.15904321057608348
  (0, 820)	0.16834609997876498
  (0, 1051)	0.17041646408818245
  (0, 372)	0.1461853719107809
  (0, 1034)	0.20268977007614375
  (0, 337)	0.20268977007614375
  (0, 1029)	0.20268977007614375
  (0, 61)	0.1526089522065723
  (0, 329)	0.179600012354591
  (0, 653)	0.15038148273564433
  (0, 223)	0.14229286920759293
  (0, 1275)	0.15572999054090678
  (0, 1013)	0.16538262692914568
  (0, 1337)	0.1518561730767219
  (0, 769)	0.22079965820477704
  (0, 365)	0.15819558942978412
  (0, 684)	0.15653937166399337
  (0, 967)	0.15414685198581074
  (0, 1356)	0.23999691322314187
  (0, 635)	0.1448563456110011
  (0, 1208)	0.14686239757223873
  (0, 384)	0.1549326006912145
  (0, 1260)	0.12131058829312995
  (0, 222)	0.13471609191396322
  (0, 143)	0.10314019107391009
  (0, 594)	0.11417655210815271
  (0, 22)	0.11343924153147902
  (0, 

## `X` is a sparse matrix which is used for representating matrices with many 0 values
 - 444 articles
 - 1456 distinct words

In [446]:
# each row has different length

len(X[0].data), len(X[1].data)

(38, 23)

In [447]:
X[0].data

array([0.16538263, 0.20878944, 0.24398561, 0.15414685, 0.15414685,
       0.15904321, 0.1683461 , 0.17041646, 0.14618537, 0.20268977,
       0.20268977, 0.20268977, 0.15260895, 0.17960001, 0.15038148,
       0.14229287, 0.15572999, 0.16538263, 0.15185617, 0.22079966,
       0.15819559, 0.15653937, 0.15414685, 0.23999691, 0.14485635,
       0.1468624 , 0.1549326 , 0.12131059, 0.13471609, 0.10314019,
       0.11417655, 0.11343924, 0.10561524, 0.11683971, 0.11568218,
       0.11683971, 0.1683461 , 0.16077958])

In [448]:
X[1].data

array([0.23680896, 0.2759931 , 0.30948595, 0.2759931 , 0.21545961,
       0.19147179, 0.24895247, 0.2652108 , 0.22290817, 0.17118446,
       0.20900738, 0.23171795, 0.2033161 , 0.17402763, 0.16353908,
       0.1936197 , 0.20900738, 0.1370062 , 0.11045416, 0.11673832,
       0.12192887, 0.18196675, 0.17863302])

### Attention
- X is a sparse matrix which each row has a differenet length
- We convert X to dense matrix and then to a tensor so all rows have the same length of 1456

In [449]:
import torch
X_torch = torch.from_numpy(X.todense())

In [450]:
X_torch

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1820, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1182, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [451]:
X_torch[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)

In [452]:
len(X_torch[1])

1456

In [453]:
df_corpus.head()

,text,label
0,PF02518 PF00512 PF13561 PF00106 PF08659 PF0050...,0
1,PF07728 PF00004 PF13165 PF13353 PF04055 PF1339...,0
2,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,0
3,PF00005 PF13561 PF00106 PF08659 PF00501 PF0010...,1
4,PF00440 PF08541 PF01551 PF01613 PF03050 PF0162...,0


### `X_torch` is  a tensor with lots of 0 values and is now ready for dataloader

---

### We need to fit `TfidfVectorizer` only on train data and then used this trained vectorizer to transform test data

In [454]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_pfam = TfidfVectorizer(stop_words='english')
tfidf_vectorizer_pfam = tfidf_vectorizer_pfam.fit(articles_train)
tfidf_vectorizer_pfam

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [455]:
type(tfidf_vectorizer_pfam)
     

sklearn.feature_extraction.text.TfidfVectorizer

### Attention!
- Because each row is not in the form of One String (article) we need to define our own `make_dataset` class

In [456]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder

class make_dataset(Dataset):
    def __init__ (self, file_name, vectorizer):
        
        if type(file_name) == str:
            df = pd.read_csv(Dataset)
        else:
            df = file_name
            
        x = df.text.values
        y = df.label.values
        
        self.X = vectorizer.transform(x)
            
        self.X = torch.from_numpy(self.X.todense())
        self.X = self.X.long()

#         sc = StandardScaler()
#         X = sc.fit_transform(x)
        
        #y = LabelEncoder().fit_transform(y)
        self.y = torch.tensor(y, dtype = torch.float32)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):    
        return self.X[idx], self.y[idx]
        

In [457]:
training_set = make_dataset(df_train, tfidf_vectorizer_pfam)
len(training_set)


310

In [458]:
testing_set = make_dataset(df_test, tfidf_vectorizer_pfam)
len(testing_set)

134

In [459]:
trainloader = torch.utils.data.DataLoader(training_set, batch_size=5, shuffle=True)
testloader = torch.utils.data.DataLoader(testing_set, batch_size = 2, shuffle = True)

In [460]:
# for testing_set we have batch_size = 2, so in each batch we have 2 examples

data_iter = iter(trainloader)

this_train_article, this_train_label = next(data_iter)
print(this_train_article.shape, this_train_label.shape)
# this_pfam.T, this_response 

torch.Size([5, 1314]) torch.Size([5])


### Attention!
- We fitted the tf-idf vectorizer only on train data (`articles_train`). Because of this, number of unique words is 1314 as you can see in the 2nd dimension of dataloader

In [461]:
# for testing_set we have batch_size = 2, so in each batch we have 2 examples

this_test_article, this_test_label = next(iter(testloader))
print(this_test_article.shape, this_test_label.shape)


torch.Size([2, 1314]) torch.Size([2])


### Attention!
- The output of `torch.utils.data.DataLoader()`has `batch_size` as the 1st elemenet. Alwyas check this because it's important when training the model.

In [462]:
import torch.nn as nn

class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 1314.
        self.fc1 = nn.Linear(1314, 512) 
        self.fc2 = nn.Linear(512, 256) 
        self.fc3 = nn.Linear(256, 64)
        self.fc_out = nn.Linear(64, 1) 
        
        #self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
#         self.batchnorm1 = nn.BatchNorm1d(1024)
#         self.batchnorm2 = nn.BatchNorm1d(256)
        
    def forward(self, inputs):
        #print("inputs", inputs.shape)# = torch.Size([5, 1314])
        
        x = torch.relu(self.fc1(inputs))
        #print("fc1", x.shape)# = torch.Size([5, 256])
        
        x = torch.relu(self.fc2(x))
        #print("fc2", x.shape) #= torch.Size([5, 64])
        
        x = torch.relu(self.fc3(x))
        #print("fc3", x.shape)# 
        
        x = self.dropout(x)
        #print("dropout", x.shape)# = torch.Size([5, 64])
               
        x = torch.sigmoid(self.fc_out(x))
        #print("fc_out", x.shape)# = torch.Size([5, 1])
        
        return x

In [463]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [464]:
from torch import optim

model = binaryClassification()
model.to(device)
print(model)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

binaryClassification(
  (fc1): Linear(in_features=1314, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=64, bias=True)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [465]:
def train_linear(model, iterator, optimizer, criterion):

    model.to(device)

    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for pfam, response in trainloader:
        
        optimizer.zero_grad()
        
        # pfam = pfam.long()
        pfam= pfam.to(device) # batch_size must be the 1st dimension so we don't use .T
        pfam = pfam.float()
     
        response = response.to(device)
        
        # output of the model is: (batch_size, 1) and we get rid of 1 with sueeze(dim=1)
        predictions = model(pfam).squeeze(1)
                
        loss = criterion(predictions, response)
        
        rounded_preds = torch.round(predictions)
        correct = (rounded_preds == response).float()
        
        acc = correct.sum() / len(correct)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [466]:
num_epochs = 20

for epoch in range(num_epochs):

    train_loss, train_acc = train_linear(model, trainloader, optimizer, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% ')

| Epoch: 01 | Train Loss: 0.757 | Train Acc: 76.13% 
| Epoch: 02 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 03 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 04 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 05 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 06 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 07 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 08 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 09 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 10 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 11 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 12 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 13 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 14 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 15 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 16 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 17 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 18 | Train Loss: 0.693 | Train Acc: 76.13% 
| Epoch: 19 | Train Loss: 0.693 | Train Acc: 7

In [467]:
def evaluate_linear(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for pfam, response in iterator:
        
            pfam= pfam.to(device) # batch_size must be the 1st dimension so we don't use .T
            pfam = pfam.float() # the model expects tensor of float type 
          
            response = response.to(device)

            # output of the model is: (batch_size, 1) and we get rid of 1 with sueeze(dim=1)
            predictions = model(pfam).squeeze(1)

            loss = criterion(predictions, response)

            rounded_preds = torch.round(predictions)
            correct = (rounded_preds == response).float()

            acc = correct.sum() / len(correct)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [468]:
N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train_linear(model, trainloader, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate_linear(model, testloader, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models\linear_model_saved_weights.pt')
    
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('----------------------------------------------------')

Epoch 1: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 2: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 3: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 4: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 5: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 6: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 7: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |  Val. Acc: 76.12%
----------------------------------------------------
Epoch 8: Train Loss: 0.693 | Train Acc: 76.13%
	 Loss: 0.693 |

---

### Clustering

In [469]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)

km.fit(X)

Initialization complete
Iteration  0, inertia 838.041
Iteration  1, inertia 405.116
Iteration  2, inertia 402.082
Iteration  3, inertia 400.004
Iteration  4, inertia 399.004
Iteration  5, inertia 398.897
Iteration  6, inertia 398.793
Iteration  7, inertia 398.667
Iteration  8, inertia 397.594
Iteration  9, inertia 397.329
Iteration 10, inertia 397.241
Iteration 11, inertia 397.227
Iteration 12, inertia 397.133
Iteration 13, inertia 397.057
Iteration 14, inertia 397.011
Converged at iteration 14: center shift 0.000000e+00 within tolerance 6.504874e-08


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=2, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=True)

In [470]:
import numpy as np
np.unique(km.labels_, return_counts=True)

(array([0, 1]), array([357,  87], dtype=int64))

---

## Creating a vocabulary

In [471]:
df_corpus.head()

,text,label
0,PF02518 PF00512 PF13561 PF00106 PF08659 PF0050...,0
1,PF07728 PF00004 PF13165 PF13353 PF04055 PF1339...,0
2,PF00072 PF00501 PF00550 PF00890 PF13450 PF0137...,0
3,PF00005 PF13561 PF00106 PF08659 PF00501 PF0010...,1
4,PF00440 PF08541 PF01551 PF01613 PF03050 PF0162...,0


In [472]:
pd.read_csv(r"DataFrames\train_pfam_corpus_p4.csv").head()

,text,label
0,PF02518 PF00512 PF00072 PF03176 PF04616 PF1356...,0
1,PF09950 PF11681 PF11863 PF13262 PF09979 PF0423...,1
2,PF02518 PF00512 PF00072 PF04616 PF13561 PF0010...,1
3,PF00072 PF00501 PF13450 PF12833 PF00165 PF1319...,1
4,PF07690 PF13302 PF00926 PF07992 PF02535 PF0008...,0


In [473]:
df_corpus.label.value_counts()

0    338
1    106
Name: label, dtype: int64

### Let's check for dupliactes

In [4]:
df_train = pd.read_csv(r"pfam_vocab/train_pfam_corpus_p4.csv")

In [5]:
def duplicate_finder(df, observation = 80):
    from collections import Counter
    
    pfams2  = []
    for txt in df["text"]:
        row = []
        txt = txt.replace('.', '')
        row.extend(word_tokenize(txt))
        pfams2.append(row)

    return Counter(pfams2[observation]).most_common()

In [6]:
duplicate_finder(df_train, 90)

[('PF01555', 1),
 ('PF06406', 1),
 ('PF10784', 1),
 ('PF03515', 1),
 ('PF01024', 1),
 ('PF03857', 1),
 ('PF14859', 1)]

In [26]:
duplicate_finder(df_corpus, 60)

[('PF13353', 1),
 ('PF00005', 1),
 ('PF00593', 1),
 ('PF07715', 1),
 ('PF00664', 1),
 ('PF13394', 1),
 ('PF13437', 1),
 ('PF03412', 1),
 ('PF13533', 1),
 ('PF12385', 1),
 ('PF01189', 1),
 ('PF13636', 1),
 ('PF17125', 1)]

### No duplicate found!
- we're good to go

In [27]:
import nltk
from nltk import word_tokenize
import torch
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

# operates on text and tokenize it using the tokenizer from nltk
TEXT = torchtext.data.Field(tokenize = word_tokenize)

In [28]:
# operates on labels and convert them to numeric values 
# in this case, Labels are already numeric so we don't need to do this conversion
LABEL = torchtext.data.LabelField(dtype = torch.float)

In [29]:
datafields = [("text", TEXT), ("labels", LABEL)]
# list of 2 tuples

In [30]:
trn, tst = torchtext.data.TabularDataset.splits(path = ".\DataFrames", 
                                                train = "train_pfam_corpus_p4.csv",
                                                test = "test_pfam_corpus_p4.csv" ,    
                                                format = 'csv',
                                                skip_header = True,
                                                fields = datafields)

In [31]:
trn[0]

In [32]:
trn[:5]

In [33]:
print(f'Number of training examples: {len(trn)}')
print(f'Number of testing examples: {len(tst)}')

Number of training examples: 310
Number of testing examples: 134


### Always perfom this check to make sure that label and text are correct 
- In the dataset:

> 1st is `text`

> 2nd is `label`

In [34]:
trn[5].text

['PF13165',
 'PF01842',
 'PF07733',
 'PF02811',
 'PF14579',
 'PF13484',
 'PF03313',
 'PF08331',
 'PF03315',
 'PF01955']

In [35]:
trn[5].labels

'0'

In [36]:
# a dict shape object which key is the label and value is a list of tokenized words
print(vars(trn.examples[1]))

{'text': ['PF09950', 'PF11681', 'PF11863', 'PF13262', 'PF09979', 'PF04233', 'PF14206', 'PF02498', 'PF09639', 'PF03374'], 'labels': '1'}


In [37]:
# build_vocab creates a vocab of the input data in this case, training data
TEXT.build_vocab(trn)

In [38]:
LABEL.build_vocab(trn)

In [39]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 1316
Unique tokens in LABEL vocabulary: 2


### The additional 2 words in `TEXT.vocab` are:
- `pad` : to fill empty spaces for each example
- `unk`: to show words which are not present in train vocab but are in test vocab

In [40]:
# most common 20 words in vocab with their frequencies
TEXT.vocab.freqs.most_common(20)

[('PF00550', 120),
 ('PF00501', 119),
 ('PF00109', 98),
 ('PF02801', 96),
 ('PF00106', 93),
 ('PF13561', 91),
 ('PF08659', 91),
 ('PF04055', 87),
 ('PF13450', 83),
 ('PF00005', 77),
 ('PF13353', 77),
 ('PF00072', 76),
 ('PF03176', 74),
 ('PF13165', 70),
 ('PF00890', 69),
 ('PF00891', 56),
 ('PF07992', 55),
 ('PF03061', 54),
 ('PF01553', 54),
 ('PF13394', 53)]

In [41]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


### Attention!
- Now data is in form ready for NLP (each row is One String) o we can use PyTorch predefined dataiterator `torchtext.data.BucketIterator`

In [42]:
batch_size = 64

train_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (trn, tst),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text), # sort based on the length of email text 
    sort_within_batch = False)

In [44]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        #self.model = model.cuda()
        
        # Creates dense embeding for each word instead of one-hot repesentation
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # hidden_dim is the output of the previous state of LSTM or RNN
        # we feed one word at a time to the LSTM layer
        # (1st word at time-step t, 2nd word at t+1, ...)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        
        # input is the last hidden state of the LSTM and output is the prediction
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text):
        
        # text: (sentence_length, batch_size)
        # Every input sentence is a list of indices of tf-idf encoded words
        embedded = self.embedding(text)
        # after applying embedded layer, every word in each sentence represented with its embedding
        # embedded: (sentence_length, batch_size, embedding_dim)
        
        # to prevent overfitting
        embedded_dropout = self.dropout(embedded)
        
        # the ouput batch of the embedding layer is the input to the RNN layer
        # the preprocessing ensures that all sentences in each batch has the same length
        output, (hidden, cell) = self.rnn(embedded_dropout)
        # output: (sentence_length, batch_size, hiden_dim)
        # output has all the hidden states for all words in a sentence
        # hidden: (1, batch_size, hiden_dim), the last hidden state for each sentence
        # cell is the output of last cell state for LSTM (Long Term Memory)
        
        # get rid of unnecessary dimension 1
        hidden_1D = hidden.squeeze(0)
        # hidden_1D: (batch_size, hiden_dim) only the last hidden state
        
        # to make sure that the output of the last hidden state is equal to hidden_1D
        assert torch.equal(output[-1, :, :], hidden_1D)
        
        return self.fc(hidden_1D)

In [48]:
input_dim = len(TEXT.vocab) # this network accepts one-hot encoded words 15002

embedding_dim = 200

hidden_dim = 256

output_dim = 1 # binary classification has only 1 neuron in the last layer

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [53]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 1e-6)
criterion = nn.BCEWithLogitsLoss()

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
def train(model, iterator, optimizer, criterion):
    

    model.to(device)

    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        batch.text = batch.text.to(device)
        batch.labels = batch.labels.to(device)
        
        # output of the model is: (batch_size, 1) and we get rid of 1 with sueeze(dim=1)
        predictions = model(batch.text).squeeze(1)
                
        loss = criterion(predictions, batch.labels)
        
        rounded_preds = torch.round(torch.sigmoid(predictions))
        correct = (rounded_preds == batch.labels).float()
        
        acc = correct.sum() / len(correct)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [58]:
num_epochs = 25

for epoch in range(num_epochs):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% ')

| Epoch: 01 | Train Loss: 0.682 | Train Acc: 68.84% 
| Epoch: 02 | Train Loss: 0.680 | Train Acc: 70.69% 
| Epoch: 03 | Train Loss: 0.682 | Train Acc: 67.53% 
| Epoch: 04 | Train Loss: 0.677 | Train Acc: 71.77% 
| Epoch: 05 | Train Loss: 0.676 | Train Acc: 75.16% 
| Epoch: 06 | Train Loss: 0.676 | Train Acc: 74.99% 
| Epoch: 07 | Train Loss: 0.676 | Train Acc: 73.88% 
| Epoch: 08 | Train Loss: 0.677 | Train Acc: 73.19% 
| Epoch: 09 | Train Loss: 0.673 | Train Acc: 75.09% 
| Epoch: 10 | Train Loss: 0.675 | Train Acc: 73.37% 
| Epoch: 11 | Train Loss: 0.671 | Train Acc: 73.33% 
| Epoch: 12 | Train Loss: 0.671 | Train Acc: 75.47% 
| Epoch: 13 | Train Loss: 0.669 | Train Acc: 75.36% 
| Epoch: 14 | Train Loss: 0.672 | Train Acc: 73.45% 
| Epoch: 15 | Train Loss: 0.666 | Train Acc: 74.90% 
| Epoch: 16 | Train Loss: 0.668 | Train Acc: 75.44% 
| Epoch: 17 | Train Loss: 0.666 | Train Acc: 75.56% 
| Epoch: 18 | Train Loss: 0.666 | Train Acc: 76.44% 
| Epoch: 19 | Train Loss: 0.664 | Train Acc: 7

In [65]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    
    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for pfam, response in iterator:
        
            pfam= pfam.to(device) # batch_size must be the 2nd dimension so we use .T
            pfam = pfam.long()
            # torch.transpose(pfam, 0, 1) it works like transpose
            response = response.to(device)

            # output of the model is: (batch_size, 1) and we get rid of 1 with sueeze(dim=1)
            predictions = model(pfam).squeeze(1)

            loss = criterion(predictions, response)

            rounded_preds = torch.round(predictions)
            correct = (rounded_preds == response).float()

            acc = correct.sum() / len(correct)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [68]:
N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/proj3_model.pt')
    
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('----------------------------------------------------')

Epoch 1: Train Loss: 0.658 | Train Acc: 75.13%
	 Loss: 0.647 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 2: Train Loss: 0.657 | Train Acc: 75.64%
	 Loss: 0.646 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 3: Train Loss: 0.655 | Train Acc: 75.87%
	 Loss: 0.645 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 4: Train Loss: 0.653 | Train Acc: 75.38%
	 Loss: 0.644 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 5: Train Loss: 0.651 | Train Acc: 75.53%
	 Loss: 0.643 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 6: Train Loss: 0.653 | Train Acc: 74.84%
	 Loss: 0.642 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 7: Train Loss: 0.651 | Train Acc: 75.56%
	 Loss: 0.641 |  Val. Acc: 83.33%
----------------------------------------------------
Epoch 8: Train Loss: 0.652 | Train Acc: 74.50%
	 Loss: 0.640 |